#### Project description
The startup is developing new apps for book lovers and the database provided is of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
book =pd.io.sql.read_sql('books',con = engine)
book

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [3]:
author =pd.io.sql.read_sql('authors',con = engine)
author

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [4]:
publisher = pd.io.sql.read_sql('publishers',con = engine)
publisher

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [5]:
rating = pd.io.sql.read_sql('ratings',con = engine)
rating

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [6]:
review = pd.io.sql.read_sql('reviews',con = engine)
review

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


####  Find the number of books released after January 1, 2000.

In [7]:
nr_books = '''
SELECT count( publication_date) as nr_books
FROM 
    books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(nr_books, con = engine)

,nr_books
0,819


Seems like a lot of books were released in a short period of time.

#### Find the number of user reviews and the average rating for each book.

In [8]:
nr_reviews = ''' 
SELECT  username,count(review_id) AS nr_reviews
FROM reviews
GROUP BY username
ORDER BY nr_reviews DESC
'''
pd.io.sql.read_sql(nr_reviews, con=engine)

,username,nr_reviews
0,susan85,29
1,sfitzgerald,28
2,martinadam,27
3,lesliegibbs,26
4,eallen,26
...,...,...
155,todd65,10
156,joseph14,10
157,dwayne09,10
158,robert00,9


The highest value is 29 and the lowest 5. A 29 user reviews is really good and the fact that there are no values lower than 5 is even better .

#### New code below : 

In [9]:
avg_rating = '''
SELECT subq.title,subq.average_rating,subq.cnt
FROM (
SELECT title,AVG(rating) as average_rating, COUNT(review_id) as cnt
FROM books
LEFT JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY title
ORDER BY cnt DESC) as subq
'''
pd.io.sql.read_sql(avg_rating, con=engine)

,title,average_rating,cnt
0,Twilight (Twilight #1),3.662500,1120
1,The Hobbit or There and Back Again,4.125000,528
2,The Catcher in the Rye,3.825581,516
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,492
4,Harry Potter and the Chamber of Secrets (Harry...,4.287500,480
...,...,...,...
994,The Natural Way to Draw,3.000000,0
995,Essential Tales and Poems,4.000000,0
996,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0
997,Leonardo's Notebooks,4.000000,0


Looks like clients love the provided books since there is no average rating lower than 3.

####  Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [10]:
publisher = '''
SELECT subq.publisher, subq.nr_books
FROM 
(SELECT publisher, COUNT(DISTINCT book_id) AS nr_books 
FROM books
LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY nr_books DESC
LIMIT 1) AS subq

'''
pd.io.sql.read_sql(publisher, con = engine)

,publisher,nr_books
0,Penguin Books,42


Penguin Books has the greatest number of books, 42. Maybe they have a lot of succes and the clients really like this kind of books.

#### Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [11]:
author = '''
SELECT subq.author, subq.avg_rating
FROM
( SELECT author, AVG(rating) as avg_rating
FROM books
LEFT JOIN ratings ON books.book_id = ratings.book_id
LEFT JOIN authors ON books.author_id = authors.author_id
GROUP BY author
HAVING COUNT( rating) >= 50 
ORDER BY avg_rating DESC
LIMIT 1) AS subq
'''
pd.io.sql.read_sql(author, con=engine)

,author,avg_rating
0,Diana Gabaldon,4.3


Diana Gabaldon is the favorite author with a very good rating of 4.3 so her books have a higher probability to be purchased than books from other authors.

#### Find the average number of text reviews among users who rated more than 50 books.

In [12]:
text_reviews = ''' SELECT avg(SUBQ.text)
FROM (
SELECT COUNT(text) AS text
FROM reviews
WHERE username in ( SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating) > 50)
GROUP BY username) AS SUBQ
'''

pd.io.sql.read_sql(text_reviews, con= engine)

,avg
0,24.333333


Book lovers like to give more text reviews than user reviews, the difference in the number of reviews been way bigger than expected.